In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 
    'mid_padding', 
                 'strf_padding', 'ext_padding', 
                 'rnd_padding', 'aug_padding', 
    "zoom_padding"]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["zoom_padding"]
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#'EC_number/archaea/rnn_conv/256rnn/'
#'EC_number/archaea/1conv/64filts_size5/'#'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task1/"
#architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
architecture = "stack_conv"

In [3]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
#model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
model = building_stackconv_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
                                       kernel_size, pool_size, final_act, folder, optimizer)
generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 26)     0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1000, 26)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 10)     270         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 1000, 10)     790         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_3 (

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, val_generator,
                        architecture, max_lenn, dict_size, batch_size,
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val, 
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1000, 26)     0                                            
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 1000, 26)     0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 1000, 10)     270         dropout_5[0][0]                  
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 1000, 10)     790         dropout_5[0][0]                  
__________________________________________________________________________________________________
conv1d_8 (

251/251 [==============================] - 6s 22ms/step - loss: 0.6073 - acc: 0.7014 - val_loss: 0.5934 - val_acc: 0.7299

Epoch 00021: val_acc improved from 0.72816 to 0.72991, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-021-0.7299.hdf5
Epoch 22/200
251/251 [==============================] - 6s 22ms/step - loss: 0.6072 - acc: 0.7044 - val_loss: 0.5939 - val_acc: 0.7313

Epoch 00022: val_acc improved from 0.72991 to 0.73131, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-022-0.7313.hdf5
Epoch 23/200
251/251 [==============================] - 6s 22ms/step - loss: 0.6059 - acc: 0.7071 - val_loss: 0.5918 - val_acc: 0.7324

Epoch 00023: val_acc improved from 0.73131 to 0.73235, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_pad

251/251 [==============================] - 6s 22ms/step - loss: 0.5452 - acc: 0.7410 - val_loss: 0.4990 - val_acc: 0.8183

Epoch 00045: val_acc improved from 0.81097 to 0.81831, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-045-0.8183.hdf5
Epoch 46/200
251/251 [==============================] - 6s 22ms/step - loss: 0.5434 - acc: 0.7433 - val_loss: 0.5070 - val_acc: 0.8215

Epoch 00046: val_acc improved from 0.81831 to 0.82145, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-046-0.8215.hdf5
Epoch 47/200
251/251 [==============================] - 6s 22ms/step - loss: 0.5369 - acc: 0.7461 - val_loss: 0.4958 - val_acc: 0.8249

Epoch 00047: val_acc improved from 0.82145 to 0.82495, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_pad

251/251 [==============================] - 6s 22ms/step - loss: 0.4856 - acc: 0.7753 - val_loss: 0.4220 - val_acc: 0.8878

Epoch 00069: val_acc improved from 0.88609 to 0.88784, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-069-0.8878.hdf5
Epoch 70/200
251/251 [==============================] - 6s 22ms/step - loss: 0.4855 - acc: 0.7735 - val_loss: 0.4266 - val_acc: 0.8892

Epoch 00070: val_acc improved from 0.88784 to 0.88924, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-070-0.8892.hdf5
Epoch 71/200
251/251 [==============================] - 6s 22ms/step - loss: 0.4861 - acc: 0.7765 - val_loss: 0.4247 - val_acc: 0.8910

Epoch 00071: val_acc improved from 0.88924 to 0.89099, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_pad

251/251 [==============================] - 6s 22ms/step - loss: 0.4288 - acc: 0.8040 - val_loss: 0.3403 - val_acc: 0.9217

Epoch 00096: val_acc improved from 0.91999 to 0.92173, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-096-0.9217.hdf5
Epoch 97/200
251/251 [==============================] - 6s 22ms/step - loss: 0.4302 - acc: 0.8057 - val_loss: 0.3413 - val_acc: 0.9231

Epoch 00097: val_acc improved from 0.92173 to 0.92313, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/post_padding/0/weights-improvement-097-0.9231.hdf5
Epoch 98/200
251/251 [==============================] - 6s 22ms/step - loss: 0.4297 - acc: 0.8051 - val_loss: 0.3432 - val_acc: 0.9224

Epoch 00098: val_acc did not improve from 0.92313
Epoch 99/200
251/251 [==============================] - 6s 22ms/step - loss: 0.4265 - acc: 0.8073 - val_loss: 0.3466 - val_

...

251/251 [==============================] - 6s 25ms/step - loss: 0.5103 - acc: 0.7682 - val_loss: 0.4586 - val_acc: 0.8739

Epoch 00063: val_acc improved from 0.87317 to 0.87386, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-063-0.8739.hdf5
Epoch 64/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5053 - acc: 0.7682 - val_loss: 0.4515 - val_acc: 0.8767

Epoch 00064: val_acc improved from 0.87386 to 0.87666, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-064-0.8767.hdf5
Epoch 65/200
251/251 [==============================] - 6s 26ms/step - loss: 0.5022 - acc: 0.7691 - val_loss: 0.4469 - val_acc: 0.8795

Epoch 00065: val_acc improved from 0.87666 to 0.87945, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_paddin

251/251 [==============================] - 6s 26ms/step - loss: 0.4484 - acc: 0.7968 - val_loss: 0.3633 - val_acc: 0.9043

Epoch 00091: val_acc improved from 0.90356 to 0.90426, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-091-0.9043.hdf5
Epoch 92/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4456 - acc: 0.7990 - val_loss: 0.3563 - val_acc: 0.9064

Epoch 00092: val_acc improved from 0.90426 to 0.90636, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-092-0.9064.hdf5
Epoch 93/200
251/251 [==============================] - 6s 26ms/step - loss: 0.4480 - acc: 0.7936 - val_loss: 0.3573 - val_acc: 0.9071

Epoch 00093: val_acc improved from 0.90636 to 0.90706, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_paddin


Epoch 00121: val_acc improved from 0.93152 to 0.93222, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-121-0.9322.hdf5
Epoch 122/200
251/251 [==============================] - 6s 26ms/step - loss: 0.4024 - acc: 0.8207 - val_loss: 0.2891 - val_acc: 0.9305

Epoch 00122: val_acc did not improve from 0.93222
Epoch 123/200
251/251 [==============================] - 6s 26ms/step - loss: 0.4010 - acc: 0.8230 - val_loss: 0.2949 - val_acc: 0.9308

Epoch 00123: val_acc did not improve from 0.93222
Epoch 124/200
251/251 [==============================] - 6s 26ms/step - loss: 0.4051 - acc: 0.8186 - val_loss: 0.2890 - val_acc: 0.9298

Epoch 00124: val_acc did not improve from 0.93222
Epoch 125/200
251/251 [==============================] - 6s 25ms/step - loss: 0.3990 - acc: 0.8243 - val_loss: 0.2886 - val_acc: 0.9322

Epoch 00125: val_acc improved from 0.93222 to 0.93222, saving model to /home/angela/pa

251/251 [==============================] - 6s 26ms/step - loss: 0.3652 - acc: 0.8383 - val_loss: 0.2426 - val_acc: 0.9434

Epoch 00155: val_acc improved from 0.94305 to 0.94340, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-155-0.9434.hdf5
Epoch 156/200
251/251 [==============================] - 6s 26ms/step - loss: 0.3736 - acc: 0.8362 - val_loss: 0.2507 - val_acc: 0.9458

Epoch 00156: val_acc improved from 0.94340 to 0.94584, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padding/2/weights-improvement-156-0.9458.hdf5
Epoch 157/200
251/251 [==============================] - 6s 26ms/step - loss: 0.3743 - acc: 0.8358 - val_loss: 0.2513 - val_acc: 0.9462

Epoch 00157: val_acc improved from 0.94584 to 0.94619, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/stack_conv/10filts_sizeJurtz/task1/ext_padd

166/251 [==================>...........] - ETA: 1s - loss: 0.3352 - acc: 0.8551